### Import Activities File

In [ ]:
## Install python-benedict library
#!pip install python-benedict

## Install for Data Visualization
#!pip install ipyleaflet
#!jupyter nbextension enable --py --sys-prefix ipyleaflet 
#
# or just use conda
#!conda install -c conda-forge ipyleaflet

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
import pandas as pd
from pathlib import Path
from os import listdir
from os.path import isfile, join
import gzip
import shutil
from benedict import benedict as bdict
from ipyleaflet import Map, Polyline


filepath = "data/"
fileOutputPath= filepath + "output/"

activityPath = filepath + "activities/"
activityOutputPath = fileOutputPath + "activities/"


def loadData():
    df = pd.read_csv(filepath+ 'activities.csv')
    data = df[df.columns[[0,1,2,3,4,5,6,10,13,14,15,16,17,18,19,20,21,22,23,26,27,31,32,41,44,45,47,50]]]

    rideTypes = ['Virtual Ride', 'Ride']
    rideData = data[data['Activity Type'].isin(rideTypes)]    
    #return rideData.dropna(subset=['Filename']).reset_index()
    return rideData

def getFilename(fileextension):
    return fileextension[:-3]

def extractAllGPXFiles():
    # Create activity files Path if it's not exists
    Path(activityOutputPath).mkdir(parents=True, exist_ok=True)
    
    # check for all files in activities folder
    activityfiles = [f for f in listdir(filepath+"activities") if isfile(join(filepath+"activities", f))]

    # Extract all files from gz zip files
    for activityzip in activityfiles:
        # Extract activity filename
        filename=getFilename(activityzip)
       
        # Extract GPX File
        with gzip.open(activityPath + activityzip, 'rb') as f_in:
            with open(activityOutputPath + filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    
    print("done!")
    
    
def getLocations(gpx_output_path,gpx_filename):
    try:
        data_source = gpx_output_path + gpx_filename
        xmldata = bdict.from_xml(data_source)
        trk_list = xmldata['gpx.trk.trkseg.trkpt']

        locs = [[float(t['@lat']),float(t['@lon'])] for t in trk_list]
    
        return locs
    except:
        return null
    

def getVirtualLocations(gpx_output_path,gpx_filename):
    try:
        data_source = gpx_output_path + gpx_filename
        xmldata = bdict.from_xml(data_source)
        trk_list = xmldata['gpx.trk.trkseg.trkpt']

        locs = [[float(t['@lat']),float(t['@lon']),float(t['ele']),t['extensions']['power'],t['time']] for t in trk_list]
    
        return locs
    except:
        return null


def showOnMap(locations):

    line = Polyline(
        locations=locations,
        color="green" ,
        fill=False
    )

    m = Map(center = (float((min(locations)[0]+max(locations)[0])/2), float(min(locations)[1]+max(locations)[1])/2), zoom =12)
    m.add_layer(line)
    return m

In [ ]:
data = loadData()

# Extract All gpx.gz files
#extractAllGPXFiles()

In [ ]:
len(data)

In [ ]:
len(getVirtualLocations(fileOutputPath,getFilename(data['Filename'][155])))

In [ ]:
fileindex = 125
showOnMap(getLocations(fileOutputPath,getFilename(data['Filename'][fileindex])))

### Print Values

In [ ]:
import datetime

def getTimeInMinute(time):
    result = time/60
    return "{:.1f}".format(result)

def getValue(value):
    return "{:.2f}".format(value)

def getSpeedkmH(speed):
    result = speed * (3600/1000)
    return "{:.2f}".format(result)
    
def getDate(strDate):
    date_time_str = strDate #'2018-06-29 08:15:27.243860'
    date_time_obj = datetime.datetime.strptime(date_time_str, '%b %d, %Y, %H:%M:%S %p')
    return date_time_obj.strftime("%b %d, %Y")

In [ ]:
data.columns

In [ ]:
import pandas as pd

def printActivityData(row):
    try:
        # print common stats
        print(" -- ", row['Activity Name'], " -- ")
        print("Activity Date:", getDate(row['Activity Date']))
        print(f"Distance: { row['Distance']} km")
        print(f"Elapsed Time: { getTimeInMinute(row['Elapsed Time'])} min")
        print(f"Moving Time: { getTimeInMinute(row['Moving Time'])} min")
        print(f"Max Speed: { getSpeedkmH(row['Max Speed'])} km/h")
        print(f"Average Speed: { getSpeedkmH(row['Average Speed'])} km/h")
        print(f"Elevation Gain: { round(row['Elevation Gain'])}")
        print(f"Elevation High: { round(row['Elevation High'])}")
        
        # print Ride Type specific stats
        if(row['Activity Type']=="Virtual Ride"):
            print(f"Weighted Average Power: { round(row['Weighted Average Power'])}")
            print(f"Power Count: { round(row['Power Count'])}")
            print(f"Max Cadence: { round(row['Max Cadence'])}")
            print(f"Average Cadence: { getValue(row['Average Cadence'])}")
            print(f"Average Watts: { round(row['Average Watts'])}")
            print(f"Calories: { round(row['Calories'])}")

    except Exception as ex:
        print("An Error Occured:", ex)
                  
def showPowerHist(virtualLocList): 
    dfSteps = pd.DataFrame(virtualLocList, columns =['Latitude', 'Longtitude','Elevation','Power','Date']) 
    dfSteps = dfSteps.astype({"Power": int})
    
    return dfSteps['Power'].plot(kind = 'hist')

def showPowerGraph(virtualLocList): 
    # Power/Time Distrubition Graph
    dfSteps = pd.DataFrame(virtualLocList, columns =['Latitude', 'Longtitude','Elevation','Power','Date']) 
    dfSteps = dfSteps.astype({"Power": int})
    powerPlot = sns.lineplot(data=dfSteps, x="Date", y="Power")
    powerPlot.set(xticklabels=[]) 
    powerPlot.set(xlabel=None)
    powerPlot.tick_params(bottom=False) 
    return powerPlot

In [ ]:
## Show details about rides

for index, row in data[-5:].iterrows():
    try:       
        # print Actvities
        printActivityData(row)

        # print Ride Type specific stats
        if(row['Activity Type']=="Virtual Ride"):
            virtualSteps = getVirtualLocations(fileOutputPath,getFilename(row['Filename']))
            virtLocs = [[t[0],t[1]] for t in virtualSteps]
            map = showOnMap(virtLocs)
            showPowerHist(virtualSteps)
            #showPowerGraph(virtualSteps)
        elif(row['Activity Type']=="Ride"):
            map = showOnMap(getLocations(fileOutputPath,getFilename(row['Filename'])))
        print(" ")
    except Exception as ex:
        print("An Error Occured:", ex)

# Print the latest power graph
# powerGraph
                  
# Print the latest map        
map

In [ ]:
# Ride Graph
sns.barplot(x="Activity Type", y="Distance", data=data, estimator=sum)

In [ ]:
data['Distance'].plot(kind = 'hist')

In [ ]:
data['Weighted Average Power'].plot(kind = 'hist')

In [ ]:
data['Calories'].plot(kind = 'hist')